In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


## Setup

In [3]:
homeDir = os.getcwd()
dataDir = homeDir + "/data/"
train_path = dataDir + "train/"
valid_path = dataDir + "valid/"
model_path = homeDir + "/data/models/"
test_path = dataDir + "test"
results_path = dataDir + "/results"

In [4]:
path = dataDir
batch_size=64

In [5]:
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [5]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In this notebook we're going to create an ensemble of models and use their average as our predictions. For each ensemble, we're going to follow our usual fine-tuning steps:

1) Create a model that retrains just the last layer
2) Add this to a model containing all VGG layers except the last layer
3) Fine-tune just the dense layers of this model (pre-computing the convolutional layers)
4) Add data augmentation, fine-tuning the dense layers without pre-computation.

So first, we need to create our VGG model and pre-compute the output of the conv layers:

In [6]:
from Vgg16 import Vgg16
from vgg16bn import Vgg16BN

In [8]:
model = Vgg16BN().model
conv_layers,fc_layers = split_at(model, Convolution2D)

In [9]:
conv_model = Sequential(conv_layers)

In [ ]:
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)
trn_features = conv_model.predict_generator(batches, batches.nb_sample)

In [ ]:
save_array(model_path + 'train_convlayer_features.bc', trn_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In the future we can just load these precomputed features:

In [10]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

We can also save some time by pre-computing the training and validation arrays with the image decoding and resizing already done:

In [ ]:
trn = get_data(path+'train')

In [ ]:
val = get_data(path+'valid')

In [ ]:
save_array(model_path+'train_data.bc', trn)
save_array(model_path+'valid_data.bc', val)

In the future we can just load these resized images:

In [11]:
trn = load_array(model_path+'train_data.bc')
val = load_array(model_path+'valid_data.bc')

Finally, we can precompute the output of all but the last dropout and dense layers, for creating the first stage of the model:

In [12]:
model.pop()
model.pop()

In [ ]:
ll_val_feat = model.predict_generator(val_batches, val_batches.nb_sample)
ll_feat = model.predict_generator(batches, batches.nb_sample)

In [ ]:
save_array(model_path + 'train_ll_feat.bc', ll_feat)
save_array(model_path + 'valid_ll_feat.bc', ll_val_feat)

In [13]:
ll_feat = load_array(model_path+ 'train_ll_feat.bc')
ll_val_feat = load_array(model_path + 'valid_ll_feat.bc')

...and let's also grab the test data, for when we need to submit:

In [ ]:
test = get_data(path+'test')
save_array(model_path+'test_data.bc', test)

In [7]:
test = load_array(model_path+'test_data.bc')

## Last layer

The functions automate creating a model that trains the last layer from scratch, and then adds those new layers on to the main model.

In [23]:
def get_ll_layers():
    return [ 
        BatchNormalization(input_shape=(4096,)),
        Dropout(0.5),
        Dense(2, activation='softmax') 
        ]

In [24]:
def train_last_layer(i):
    ll_layers = get_ll_layers()
    ll_model = Sequential(ll_layers)
    ll_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    ll_model.optimizer.lr=1e-5
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), nb_epoch=12)
    ll_model.optimizer.lr=1e-7
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), nb_epoch=1)
    ll_model.save_weights(model_path+'ll_bn' + i + '.h5')

    vgg = Vgg16BN()
    model = vgg.model
    model.pop(); model.pop(); model.pop()
    for layer in model.layers: layer.trainable=False
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    ll_layers = get_ll_layers()
    for layer in ll_layers: model.add(layer)
    for l1,l2 in zip(ll_model.layers, model.layers[-3:]):
        l2.set_weights(l1.get_weights())
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.save_weights(model_path+'bn' + i + '.h5')
    return model

## Dense model

In [25]:
def get_conv_model(model):
    layers = model.layers
    last_conv_idx = [index for index,layer in enumerate(layers) 
                         if type(layer) is Convolution2D][-1]

    conv_layers = layers[:last_conv_idx+1]
    conv_model = Sequential(conv_layers)
    fc_layers = layers[last_conv_idx+1:]
    return conv_model, fc_layers, last_conv_idx

In [26]:
def get_fc_layers(p, in_shape):
    return [
        MaxPooling2D(input_shape=in_shape),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(2, activation='softmax')
        ]

In [27]:
def train_dense_layers(i, model):
    conv_model, fc_layers, last_conv_idx = get_conv_model(model)
    conv_shape = conv_model.output_shape[1:]
    fc_model = Sequential(get_fc_layers(0.5, conv_shape))
    for l1,l2 in zip(fc_model.layers, fc_layers): 
        weights = l2.get_weights()
        l1.set_weights(weights)
    fc_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                     metrics=['accuracy'])
    fc_model.fit(trn_features, trn_labels, nb_epoch=2, 
         batch_size=batch_size, validation_data=(val_features, val_labels))

    gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.05, zoom_range=0.05,
       channel_shift_range=10, height_shift_range=0.05, shear_range=0.05, horizontal_flip=True)
    batches = gen.flow(trn, trn_labels, batch_size=batch_size)
    val_batches = image.ImageDataGenerator().flow(val, val_labels, 
                      shuffle=False, batch_size=batch_size)

    for layer in conv_model.layers: layer.trainable = False
    for layer in get_fc_layers(0.5, conv_shape): conv_model.add(layer)
    for l1,l2 in zip(conv_model.layers[last_conv_idx+1:], fc_model.layers): 
        l1.set_weights(l2.get_weights())

    conv_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                       metrics=['accuracy'])
    #conv_model.save_weights(model_path+'no_dropout_bn' + i + '.h5')
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=1,
                             validation_data=val_batches, nb_val_samples=val_batches.N)
    conv_model.save_weights(model_path+'m1' + i + '.h5')
    for layer in conv_model.layers[16:]: layer.trainable = True
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=1, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)
    conv_model.save_weights(model_path+'m2' + i + '.h5')

    #conv_model.optimizer.lr = 1e-7
    #conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=3, 
                           # validation_data=val_batches, nb_val_samples=val_batches.N)
    #conv_model.save_weights(model_path + 'aug' + i + '.h5')

## Build ensemble

In [ ]:
for i in range(5):
    i = str(i)
    model = train_last_layer(i)
    train_dense_layers(i, model)

Train on 23000 samples, validate on 2000 samples
Epoch 1/12
23000/23000 [==============================] - 3s - loss: 0.5398 - acc: 0.8076 - val_loss: 2.3753 - val_acc: 0.3735
Epoch 2/12
23000/23000 [==============================] - 3s - loss: 0.4618 - acc: 0.8607 - val_loss: 2.5498 - val_acc: 0.3760
Epoch 3/12
23000/23000 [==============================] - 3s - loss: 0.4490 - acc: 0.8702 - val_loss: 2.4868 - val_acc: 0.3730
Epoch 4/12
23000/23000 [==============================] - 3s - loss: 0.4355 - acc: 0.8740 - val_loss: 2.5565 - val_acc: 0.3735
Epoch 5/12
23000/23000 [==============================] - 3s - loss: 0.4260 - acc: 0.8762 - val_loss: 2.5125 - val_acc: 0.3715
Epoch 6/12
23000/23000 [==============================] - 3s - loss: 0.4213 - acc: 0.8776 - val_loss: 2.4618 - val_acc: 0.3720
Epoch 7/12
23000/23000 [==============================] - 3s - loss: 0.4178 - acc: 0.8787 - val_loss: 2.5337 - val_acc: 0.3695
Epoch 8/12
23000/23000 [==============================] - 3s -

In [28]:
for i in range(2):
    i = str(i)
    model = train_last_layer(i)
    train_dense_layers(i, model)

Train on 23000 samples, validate on 2000 samples
Epoch 1/12
23000/23000 [==============================] - 4s - loss: 0.6407 - acc: 0.7653 - val_loss: 2.2491 - val_acc: 0.3835
Epoch 2/12
23000/23000 [==============================] - 4s - loss: 0.4806 - acc: 0.8523 - val_loss: 2.5045 - val_acc: 0.3750
Epoch 3/12
23000/23000 [==============================] - 4s - loss: 0.4605 - acc: 0.8632 - val_loss: 2.5822 - val_acc: 0.3735
Epoch 4/12
23000/23000 [==============================] - 4s - loss: 0.4363 - acc: 0.8710 - val_loss: 2.4512 - val_acc: 0.3720
Epoch 5/12
23000/23000 [==============================] - 4s - loss: 0.4262 - acc: 0.8757 - val_loss: 2.4646 - val_acc: 0.3715
Epoch 6/12
23000/23000 [==============================] - 4s - loss: 0.4127 - acc: 0.8785 - val_loss: 2.4646 - val_acc: 0.3710
Epoch 7/12
23000/23000 [==============================] - 4s - loss: 0.4142 - acc: 0.8790 - val_loss: 2.4168 - val_acc: 0.3705
Epoch 8/12
23000/23000 [==============================] - 4s -

## Combine ensemble and test

In [ ]:
ens_model = vgg_ft(2)
for layer in ens_model.layers: layer.trainable=True

In [8]:
vgg = Vgg16BN()
model = vgg.model
model.pop()
model.add(Dense(2, activation='softmax'))
for layer in model.layers: layer.trainable=True

In [9]:
def get_ens_pred(arr, fname):
    ens_pred = []
    for i in [0,1,3,4]:
        i = str(i)
        model.load_weights('{}{}{}.h5'.format(model_path, fname, i))
        preds = model.predict(arr, batch_size=64)
        ens_pred.append(preds)
    return ens_pred

In [ ]:
def get_comb_pred(arr):
    ens_pred = []
    for i in ['m10','m11','m20','m21']:
        model.load_weights('{}{}.h5'.format(model_path, i))
        preds = model.predict(arr, batch_size=64)
        ens_pred.append(preds)
    return ens_pred

In [27]:
val_pred2 = get_ens_pred(val, 'aug')

NameError: name 'val' is not defined

In [ ]:
val_avg_preds2 = np.stack(val_pred2).mean(axis=0)

In [ ]:
categorical_accuracy(val_labels, val_avg_preds2).eval()

In [10]:
test_pred = get_ens_pred(test, 'aug')

In [11]:
test_avg_preds = np.stack(test_pred).mean(axis=0)

In [13]:
filenames = test_filenames
isdog = test_avg_preds[:,1]
isdog = isdog.clip(min=0.005, max=0.995)
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
subm = np.stack([ids,isdog], axis=1)
subm[:5]
%cd $dataDir
submission_file_name = 'submission_ensemble3.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')
from IPython.display import FileLink
%cd ../
FileLink('data/'+submission_file_name)

/home/ubuntu/KaggleCompetitions/cats_dogs_redux/data
/home/ubuntu/KaggleCompetitions/cats_dogs_redux


/home/ubuntu/KaggleCompetitions/cats_dogs_redux/data/submission_ensemble3.csv